<h1>Capstone Week 3 - Part 1</h1>

In [20]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

<h1>Lets get Wikipedia Content first</h1>

In [2]:
wikipedia_link = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
results = requests.get(wikipedia_link).text
wikipedia_content = BeautifulSoup(results, 'lxml')    

<h1>Now lets extract the data and create our DataFrame</h1>

In [3]:
column_names = ["PostalCode","Borough","Neighbourhood"]
df = pd.DataFrame(columns=column_names)
rows = (wikipedia_content.find("table").find_all("tr"))
for row in rows[1:]:
    postal = row.find_all("td")[0].text.replace("\n","") # Skip new lines
    borough = row.find_all("td")[1].text.replace("\n","")
    neighbor = row.find_all("td")[2].text.replace("\n","")
    
    # We skip rows containing Not Assigned Borough or Neighbor
    if ("NOT ASSIGNED" == borough.upper()):
        continue
    
    df = df.append({'PostalCode': postal,
                    'Borough': borough,
                    'Neighbourhood' : neighbor}, ignore_index=True)
                    
print(df.head())
print(df.shape)

  PostalCode           Borough     Neighbourhood
0        M3A        North York         Parkwoods
1        M4A        North York  Victoria Village
2        M5A  Downtown Toronto      Harbourfront
3        M5A  Downtown Toronto       Regent Park
4        M6A        North York  Lawrence Heights
(212, 3)


<h1>Since more than one neighbourhood can exist in one postal code area, lets combine the rows and separate neighbourhoods by comma. I assume, Borough is the same so I group only by PostalCode</h1>

In [10]:
aggregation_functions = {'Borough': 'first', 'Neighbourhood': lambda x: ', '.join(x)}
df_new = df.groupby(df['PostalCode']).aggregate(aggregation_functions).reset_index()
df_new.head()

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [23]:
# Just a quick check to confirm line 9 (M5V) in instructions
print(np.array(df_new[df_new.PostalCode == 'M5V']['Neighbourhood']))

[ 'CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara']


In [26]:
# Confirm M5A
print(np.array(df_new[df_new.PostalCode == 'M5A']['Neighbourhood']))

['Harbourfront, Regent Park']


In [27]:
print ('Shape of cleaned data:',df_new.shape)

Shape of cleaned data: (103, 3)
